# EML4 - Aula 04 - Processamento de Massivo de Dados

Principais links:
* [Spark docs](https://spark.apache.org/docs/latest/)
* [Dataset Adult Pre-processed](https://drive.google.com/file/d/1WLKTZUnhcVVwwBDvwe84TRSKdyHd-0uh/view?usp=sharing)
  * Dados do censo de 1994
  * [Link Repositório Original](https://archive.ics.uci.edu/ml/datasets/Adult)
    *  Formato: csv

Nos notebooks da Databricks, o Spark Session fica disponivel pela variável global `spark`

In [1]:
spark

### Upload do Dataset
* Faça upload do conjunto de dados "Adult" preprocessado que disponibilizamos junto ao notebook da aula
* Obtenha o caminho para DFS onde foi armazenado o arquivo e coloque na variável "dataset_location" no próxima célula

In [ ]:
# Cole aqui o caminho gerado para o seu dataset
dataset_location = "dbfs:/FileStore/shared_uploads/naldi@ufscar.br/adult_preprocessed.data"

Uma fez feito o upload, o arquivo será divido, replicado e colocado em um DFS (Distributed File System). Para acessá-lo precisaremos utilizar uma estrutura que o abstraia.

![Unified Engine](https://files.training.databricks.com/images/105/unified-engine.png)

Inicialmente, a estrutura básica do Spark foi definida como RDD

* **R**esilient: Fault-tolerant
* **D**istributed: Across multiple nodes
* **D**ataset: Collection of partitioned data

RDDs são imutáveis quando criados e mantém registros de sua linhagem para recuperação de falhas! Estas estruturas permitem o acesso ao conjunto de dados distribuído, mas utilizam memória principal sempre que possível. Outras abstrações do Spark utilizam um RDD para acesso aos dados armazenados.

Contudo, existem abstrações de alto nível, mais fáceis de usar e que possibilitam um resultado com melhor performance como os DataFrames!

![RDD vs DataFrames](https://files.training.databricks.com/images/105/rdd-vs-dataframes.png)

### DataFrame
Através da sessão, vamos importar o dataset para um DataFrame

In [ ]:
dataset = spark.read.format('csv') \
               .option('inferSchema', True) \
               .option('header', False) \
               .option('sep', ',') \
               .load(dataset_location)

display(dataset)

É possível obter o esquema de dados de um DataFrame usando o método `printSchema()`.

In [ ]:
dataset.printSchema()

O data set foi carregado como um DataFrame.

Uma vez que o dataset não possuía header, não foi possível atribuir o nome das colunas automaticamente. 

Agora, vamos atribuir manualmente o nome às colunas

In [ ]:
dataset1 = dataset \
            .withColumnRenamed('_c0', 'idade') \
            .withColumnRenamed('_c1', 'classe_trabalho') \
            .withColumnRenamed('_c2', 'final_weight') \
            .withColumnRenamed('_c3', 'escolaridade') \
            .withColumnRenamed('_c4', 'escolaridade_num') \
            .withColumnRenamed('_c5', 'estado_civil') \
            .withColumnRenamed('_c6', 'ocupacao') \
            .withColumnRenamed('_c7', 'relacionamento_householder') \
            .withColumnRenamed('_c8', 'raca') \
            .withColumnRenamed('_c9', 'sexo') \
            .withColumnRenamed('_c10', 'ganho_capital') \
            .withColumnRenamed('_c11', 'perda_capital') \
            .withColumnRenamed('_c12', 'jornada_trabalho') \
            .withColumnRenamed('_c13', 'nacionalidade') \
            .withColumnRenamed('_c14', 'renda_anual')

display(dataset1)

In [ ]:
dataset1.printSchema()

In [ ]:
# Selecionando colunas específicas
dataset1.select('idade', 'nacionalidade').show(5)

In [ ]:
# Operando colunas
dataset1.select( \
                'ganho_capital', 'perda_capital', \
                dataset1['ganho_capital'] - dataset1['perda_capital'] \
               ).show(5)

In [ ]:
# Utilizando expressões
from pyspark.sql.functions import expr
dataset1.select(\
     'ganho_capital', 'perda_capital',\
     expr('ganho_capital - perda_capital as capital_liquido')\
).show(5)

In [ ]:
# Utilizando expressões com agregação
dataset1.selectExpr('avg(idade)').show()

#### Exercício 1
Obtenha o valor máximo de capital líquido (ganho de capital - perda de capital)

In [ ]:
#Faça aqui o código do exercício 1

In [ ]:
# Filtrando registros
dataset1.filter('idade < 30').show(2)
dataset1.where('idade > 30').show(2)

In [ ]:
# Filtrando registros
dataset1\
  .filter('idade < 30')\
  .where(dataset1['estado_civil'] == 'Never-married')\
  .filter(dataset1['nacionalidade'] != 'United-States')\
  .show(5)

In [ ]:
#ordenando por idade e filtrando por sexo
dataset1\
  .sort(dataset1['idade'].desc())\
  .filter(dataset1['sexo'] == 'Male')

#esse código tem boa performance?  

Porque os resultados não são mostrados na célula acima?

Ordenação e filtros são transformações, que são avaliadas de forma *lazy* pelo Spark.

Isso gera várias vantagens, dentre elas: impede a leitura desnecessária do conjunto de dados; facilita o paralelismo; possibilita otimização!
  
Para saber mais sobre o otimizador do Spark **Catalyst** leia [esse blog!](https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html)
  
![Catalyst](https://files.training.databricks.com/images/105/catalyst-diagram.png)

#### Exercício 2

Obtenha a idade média das pessoas viúvas com jornada de trabalho acima de 20 horas semanais

In [ ]:
#Faça aqui o código do exercício 2

      

In [ ]:
# Obtendo valores distintos
dataset1.select('estado_civil').distinct().show()

#### Exercício 3
Obtendo valores distintos para a combinação de sexo e raça para pessoas com idade acima de 60 anos

In [ ]:
#Faça aqui o código do exercício 3

In [ ]:
# Obtendo valores agregados, dado um agrupamento
from pyspark.sql.functions \
    import count, sum, max, min, avg

dataset1 \
    .groupBy('sexo') \
    .agg(avg('idade')) \
    .show()

In [ ]:
# Obtendo valores agregados, dado vários agrupamentos
dataset1 \
    .groupBy('sexo', 'estado_civil') \
    .agg(avg('idade'), max('idade'), min('idade')) \
    .show()

#### Exercício 4
Obtenha a média de capital líquido (ganho de capital - perda de capital) por escolaridade de pessoas com idade acima de 30 anos

In [ ]:
#Faça aqui o código do exercício 4

In [ ]:
# Obtendo as 5 ocupações com maior ganho de capital médio

from pyspark.sql.functions import asc, desc
dataset1 \
    .groupBy('ocupacao') \
    .agg(avg('ganho_capital').alias('ganho_medio')) \
    .orderBy(desc('ganho_medio')) \
    .limit(5) \
    .show()

#### Exercício 5
Obtenha a combinação de escolaridade e ocupação com menor jornada de trabalho média que tenham renda maior de 50 mil dólares por ano

In [ ]:
#Faça aqui o código do exercício 5

In [ ]:
# Usando collect, take ou first para obter os valores calculados

resultado =  dataset1 \
    .groupBy('estado_civil')\
    .agg(avg('idade')) \
    .collect()

print(resultado)

print(resultado[0]['avg(idade)'])

In [ ]:
#Obter a idade média por ocupação, substituindo o agrupamento de dados por um laço iterativo (não é uma estratégia recomendada)

ocupacoes = dataset1.select('ocupacao').distinct().collect()

print(ocupacoes)

for i in ocupacoes:
  ocupacao = i['ocupacao']
  
  idade_media = dataset1 \
      .filter(dataset1['ocupacao'] == ocupacao) \
      .selectExpr('avg(idade)') \
      .first()
  
  print("%s: %f" % (ocupacao, idade_media['avg(idade)']))

### API SQL

In [ ]:
# Para utilizar a API, vamos disponibilizar nosso DataFrame em formato de tabela para a API SQL

temp_table_name = 'dataset'
dataset1.createOrReplaceTempView(temp_table_name)

In [ ]:
# A partir de agora, é possível fazer consultas SQL sobre a tabela recém-criada (dataset).

spark.sql('''
  select sexo, avg(idade) as idade_media
  from dataset
  group by sexo
''').show()

In [ ]:
%sql
-- Consulta a partir de uma célula SQL
select sexo, avg(idade) as idade_media
from dataset
group by sexo

#### Exercício 6
O atributo final_weight representa quantas vezes uma determinada leitura do censo é repetida, ou seja, o peso daquela linha no conjunto de dados. Itere sobre a lista das 5 maiores nacionalidades (baseado na soma do final weight) e para cada uma, obtenha os três estados civis com maior jornada de trabalho médio. Utilize a API SQL.

In [ ]:
#Faça aqui o código do exercício 6